# Watermark of File

In [1]:
import numpy as np
import time
import copy

# https://github.com/rasbt/watermark#installation-and-updating
# need to pip install watermark
import watermark 

In [2]:
%load_ext watermark

In [3]:
%watermark

Last updated: 2022-04-16T21:24:55.244140-07:00

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.31.1

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 21.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit



In [4]:
%watermark --iversions

numpy    : 1.21.0
watermark: 2.3.0



# Backgammon

Backgammon is such a wonderful game. I loved it so much that I have taken the time to write up the things I have learned about the game, and have built the Monte Carlo rollouts for these statistics to show some level of real-world expectation. I will hook up all of my old games to this file once I download them all, and this will give you a sense of how this works from real games. 

### Terminology

For a full list of terminology, you should visit the US Backgammon Federation [Glossary of Terms](https://usbgf.org/backgammon-glossary/). A highlight of the main terms we will use are given below in alphabetical order:

> **Checker:** One of the 30 playing pieces used in the game of backgammon. Each player has 15 checkers.
> 
> **Game:** The result of one player getting all of their checkers off of the board. The player will win a predetermined number of points not accounting for modifiers (gammon, backgammon, cubing, ...).
> 
> **Match:** To play a series of backgammon games up to a pre-assigned number of points. The first player to reach the number of specified points is declared the winner of the match.
>
> **On the Bar:** When your checker is hit and is waiting to re-enter, you are said to be “on the bar”.
> 
> **Pip:** 1 of the 24 points or triangles on a backgammon board.
> 
> **Pip Count:** The number of spaces remaining for each player to bear all his checkers off the board — the totals of which are called the current Pip Count.
> 
> **Prime:** A number of consecutive points, usually four, five or six in a row, that are made with the intention of blocking an opponent from escape. The best prime is when you hold six consecutive points, called Full Prime.
> 
> **Roll:** A toss or throw of the dice.
> 
> **Rollout:** To analyze a position by playing it out many times, either manually or with a computer program.
> 
> 

### My Favorite Fact about Backgammon: Ties Don't Exist!

Unlike many other stategy games, backgammon contains no game or match level stalemate/tie situations! This blew my mind once I sat down to think about it. 

The intuition for no match ties is pretty simple. Since a match is simply a collection of games, we can think of those games as ending in a stalemate/tie or not. If they tie, then you could effectively 'waste' the game via a replay. Otherwise, a player wins at least one point and the match continues. If we keep playing games in this fashion, then the match shouldn't tie because one player will reach the rquired number of points before the other. But this assumes that the players do not enter an indefinite stalemate/tie situation. So let's think through the stalemate/tie situation at the game level. If we show this doesn't happen, then we know that matches will not result in a tie either.

When we think about the game level, there are a few immediate observations that are needed:

1. The only way to prevent an opponent’s checker from moving forward in all circumstances is a 6-point prime. This follows because the largest roll of a die is 6, and combining the dice to move further than 6 points requires that each die be moved independently, meaning you must be able to move 1, 2, 3, 4, 5, or 6 points.
2. An established prime of any length will not hold given sufficiently many rolls. This follows because a player is compelled to move when possible. This leads to primes of progressively shorter length and/or gaps in a prime structure. Both of these situations eventually allow opposing runners. So priming an opponent is insufficient to produce a stalemate on its own.
3. A player’s position at any point time is a collection of primes scattered around the board.
4. The only way to prevent an eventual break down in priming structure is if a prime is set and that player is prevented from moving. The only way this can occur is if at least one of their checkers is not on the board.

These observations, taken together, mean that we only need disprove the existence of a position with the following qualities:

1. both home boards are closed, and
2. both players have at least one checker on the bar.

Let’s analyze how this would occur. To send your opponent to the bar, all of your checkers must be on the board for the hit to occur. If this position (post hit) resulted in an open board (i.e, your home board has at least one point with zero or one checkers), then a move is possible, so we are not in a stalemate position. So let’s assume the resulting position is closed. Your home board being closed means your opponent does not have all of their checkers on the board, and they cannot get any additional checkers on the board during their turn. This implies that they cannot send you to the bar at this move. This means one player is closed out before another, and the observations above mean a move is still possible. Thus, our hypothetical position is not possible, so a stalemate position at the game level is not possible. This also implies that the match level stalemate/tie is also not possible!

# Backgammon Probability 101

The purpose of this workbook is to demonstrate backgammon probabilities using Monte Carlo techniques. Before we go further, let's set up a consistent number of simulations to perform in each test. Additionally, let's set up functions to track side values possible pairs.

In [5]:
number_of_simulations = 2500000

In [6]:
def side_values():
    '''
    Get the six sides of a standard die. Returns the ordered list 
    [1, 2, 3, 4, 5, 6].
    '''
    
    return [i for i in range(1,7,1)] + [None]

In [7]:
def pair_values():
    '''
    Get all possible pairings of two six-sided dice. Face-up pairs are listed 
    as an integer where the first digit is the top face of die one and the 
    second digit is the top face of die two. Returns the ordered list 
    [11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 31, 32, 33, 34, 35, 36,
     41, 42, 43, 44, 45, 46, 51, 52, 53, 54, 55, 56, 61, 62, 63, 64, 65, 66]
    '''
    
    pv_pairs = []
    for i in range(1,7,1):
        for j in range(1,7,1):
            pv_pairs += [i*10+j]
            
    return pv_pairs + [None]

In [8]:
def pip_counts():
    '''
    Get the list of possible pip counts of rolling two six-sided dice. Returns
    the list [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 18, 20, 24, None].
    '''
    
    p_counts = []
    for i in range(1,7,1):
        for j in range(1,7,1):
            if i == j:
                p_counts += [i, 2*i, 3*i, 4*i]
            else:
                p_counts += [i+j]
                
    return sorted(set(p_counts)) + [None]

### Rolling a Particular Number

Backgammon is played with pair of fair six-sided dice, where each die is labeled with the numbers 1, 2, 3, 4, 5, or 6 on its' sides, and no number repeats itself. This means the probability of expecting any one number is 1/6, or approximately 0.1666666667. Let's confirm this with a Monte Carlo rollout.

In [9]:
def probability_of_single_number(simulation_number):
    '''
    The Monte Carlo simulation of rolling a six-side fair die compared to
    theoretical value. Prints a table of observed probability vs. theoretical 
    probability for each face value.
    '''
    
    tic = time.perf_counter()
    
    posn_sv = side_values()
    orig_list = [0, None] # Occurrence Count, Observed Probability
    posn_vals = [copy.deepcopy(orig_list) for i in range(0, len(posn_sv))]
    roll_dict = dict(zip(posn_sv, posn_vals))
    roll_dict.pop(None)
    
    for sim in range(0, simulation_number, 1):
        die_roll = np.random.randint(1,7)
        roll_dict[die_roll][0] += 1
        try:
            posn_sv.remove(die_roll)
        except:
            continue
    
    for num in range(1, 7, 1):
        roll_dict[num][1] = roll_dict[num][0] / simulation_number
        
    dash = '-' * 71
    
    print('')
    print('')
    print('Probability of Rolling a Single Number')
    print('(One Die Rolled at a Time)')
    print('')
    print('')
    print(f'Roll{" ":5}Occurences{" ":5}Observed{" ":6}Expected{" ":6}Absolute Difference', end='')
    print('')
    print(dash)
    for k, v in roll_dict.items():
        print(f'{k:<9}{v[0]:<15}{round(v[1],7):<14,.07f}{round(1/6,7):<14,.07f}{abs(round(1/6-v[1],7)):>9,.07f}')
    print('')
    print('')
    print('Total Rolls: {:,}'.format(simulation_number))
    print('Nonoccurence Face Values: {}'.format(posn_sv))
    print('Total Time: {0:.2f} seconds'.format(time.perf_counter()-tic)) 
    print('')
    print('')
    
    return None
    
probability_of_single_number(number_of_simulations)



Probability of Rolling a Single Number
(One Die Rolled at a Time)


Roll     Occurences     Observed      Expected      Absolute Difference
-----------------------------------------------------------------------
1        417436         0.1669744     0.1666667     0.0003077
2        415791         0.1663164     0.1666667     0.0003503
3        417476         0.1669904     0.1666667     0.0003237
4        416824         0.1667296     0.1666667     0.0000629
5        416053         0.1664212     0.1666667     0.0002455
6        416420         0.1665680     0.1666667     0.0000987


Total Rolls: 2,500,000
Nonoccurence Face Values: [None]
Total Time: 6.70 seconds




### Probability of Particular Pairs of Numbers

Because we roll a pair of numbers, we can think of the resulting table being bidirectional, where the vertical numbers are from one die, and horizontal numbers are from the other die. Each pairing (d1,d2) is a pairing of the values from die 1 and die 2, respectively. For instance, the roll die 1 = 3 and die 2 = 4 would be represented by (3,4). For simplicity, we can concatenate the numbers like d1d2. For instance, the roll die 1 = 3 and die 2 = 4 would be represented by 34. 

The sample space of dice outcomes is given below.

|     | 1   | 2   | 3   | 4   | 5   | 6   |
- --- | --- | --- | --- | --- | --- | --- |
| 1 | (1,1) = 11 | (1,2) = 12 | (1,3) = 13 | (1,4) = 14 | (1,5) = 15 | (1,6) = 16 |
| 2 | (2,1) = 21 | (2,2) = 22 | (2,3) = 23 | (2,4) = 24 | (2,5) = 25 | (2,6) = 26 |
| 3 | (3,1) = 31 | (3,2) = 32 | (3,3) = 33 | (3,4) = 34 | (3,5) = 35 | (3,6) = 36 |
| 4 | (4,1) = 41 | (4,2) = 42 | (4,3) = 43 | (4,4) = 44 | (4,5) = 45 | (4,6) = 46 |
| 5 | (5,1) = 51 | (5,2) = 52 | (5,3) = 53 | (5,4) = 54 | (5,5) = 55 | (5,6) = 56 |
| 6 | (6,1) = 61 | (6,2) = 62 | (6,3) = 63 | (6,4) = 64 | (6,5) = 65 | (6,6) = 66 |

Since the dice are symmetric, we can think of the dice table as being lower triangular, as shown below, in which case the larger of the two dice values is listed first and the smaller of the two dices values is listed second. For instance, die 1 = 3 and die 2 = 4 would be written as 43, and die 1 = 4 and die 2 = 3 would also be written as 43. 

|     | 1   | 2   | 3   | 4   | 5   | 6   |
- --- | --- | --- | --- | --- | --- | --- |
| 1 | (1,1) = 11 |  |  |  |  |  |
| 2 | (2,1) = 21 | (2,2) = 22 |  |  |  |  |
| 3 | (3,1) = 31 | (3,2) = 32 | (3,3) = 33 |  |  |  |
| 4 | (4,1) = 41 | (4,2) = 42 | (4,3) = 43 | (4,4) = 44 |  |  |
| 5 | (5,1) = 51 | (5,2) = 52 | (5,3) = 53 | (5,4) = 54 | (5,5) = 55 | |
| 6 | (6,1) = 61 | (6,2) = 62 | (6,3) = 63 | (6,4) = 64 | (6,5) = 65 | (6,6) = 66 |

Since pairings listed in the first table above are unique, the probability of any one of them occurring is 1/36, or approximately 0.0277777777. If we view the dice from the point of view of the second table, then we could also confirm that any off-diagonal entry (non-doubles) should occur at twice the rate of any diagonal entry (doubles) because of symmetry. Let's confirm these with a Monte Carlo rollout.

In [10]:
def probability_of_paired_numbers(simulation_number):
    '''
    The Monte Carlo simulation of rolling a pair of six-side fair die compared
    to theoretical value. Prints a table of observed probability vs. 
    theoretical probability of each ordered roll.
    '''
    
    tic = time.perf_counter()
    
    popn_pv = pair_values()
    orig_list = [0, None] # Occurrence Count, Observed Probability
    popn_vals = [copy.deepcopy(orig_list) for i in range(0, len(popn_pv))]
    roll_dict = dict(zip(popn_pv, popn_vals))
    roll_dict.pop(None)
        
    for sim in range(0, simulation_number, 1):
        first_number = np.random.randint(1,7)
        second_number = np.random.randint(1,7)
        roll = first_number * 10 + second_number
        roll_dict[roll][0] += 1
        try:
            popn_pv.remove(roll)
        except:
            continue
    
    for roll in roll_dict.keys():
        roll_dict[roll][1] = roll_dict[roll][0] / simulation_number
        
    dash = '-' * 71
    
    print('')
    print('')
    print('Probability of Rolling a Given Ordered Pair')
    print('(Two Dice Rolled at a Time)')
    print('')
    print('')
    print(f'Roll{" ":6}Occurences{" ":5}Observed{" ":6}Expected{" ":6}Absolute Difference', end='')
    print('')
    print(dash)
    for k, v in roll_dict.items():
        print(f'{k:<9}{v[0]:<15}{round(v[1],7):<14,.07f}{round(1/36,7):<14,.07f}{abs(round(1/36-v[1],7)):>9,.07f}')
    print('')
    print('')
    print('Total Rolls: {:,}'.format(simulation_number))
    print('Nonoccurence Ordered Pairs: {}'.format(popn_pv))
    print('Total Time: {0:.2f} seconds'.format(time.perf_counter()-tic)) 
    print('')
    print('')
    
    return None
    
probability_of_paired_numbers(number_of_simulations)



Probability of Rolling a Given Ordered Pair
(Two Dice Rolled at a Time)


Roll      Occurences     Observed      Expected      Absolute Difference
-----------------------------------------------------------------------
11       69342          0.0277368     0.0277778     0.0000410
12       69522          0.0278088     0.0277778     0.0000310
13       69770          0.0279080     0.0277778     0.0001302
14       69500          0.0278000     0.0277778     0.0000222
15       69070          0.0276280     0.0277778     0.0001498
16       69410          0.0277640     0.0277778     0.0000138
21       69488          0.0277952     0.0277778     0.0000174
22       69013          0.0276052     0.0277778     0.0001726
23       69125          0.0276500     0.0277778     0.0001278
24       69859          0.0279436     0.0277778     0.0001658
25       69321          0.0277284     0.0277778     0.0000494
26       69065          0.0276260     0.0277778     0.0001518
31       69391          0.0277564  

In [11]:
def probability_of_off_diag_vs_diag_numbers(simulation_number):
    '''
    The Monte Carlo simulation for the combinations of rolling two six-sided
    dice. Because we are looking at combinations and not permutations, rolls
    are 'binned'; for instance, 12 and 21 both get put in the bin 12. Prints 
    two tables of observed probability vs. theoretical probability, one for
    doubles and one for non-doubles.
    '''
    
    tic = time.perf_counter()
    
    roll_dict = {11:[0,None], 
                 22:[0,None], 
                 33:[0,None],
                 44:[0,None], 
                 55:[0,None], 
                 66:[0,None],
                 21:[0,None], 
                 31:[0,None], 
                 32:[0,None], 
                 41:[0,None], 
                 42:[0,None], 
                 43:[0,None], 
                 51:[0,None], 
                 52:[0,None], 
                 53:[0,None], 
                 54:[0,None], 
                 61:[0,None], 
                 62:[0,None], 
                 63:[0,None], 
                 64:[0,None], 
                 65:[0,None]}
    nonoccurence_vals = list(roll_dict.keys()) + [None]
        
    for sim in range(0, simulation_number, 1):
        first_number = np.random.randint(1,7)
        second_number = np.random.randint(1,7)
        if first_number < second_number:
            first_number, second_number = second_number, first_number
        roll = first_number * 10 + second_number
        roll_dict[roll][0] += 1
        try:
            nonoccurence_vals.remove(roll)
        except:
            continue
            
    for roll in roll_dict.keys():
        roll_dict[roll][1] = roll_dict[roll][0] / simulation_number
        
    dash = '-' * 79
    
    print('')
    print('')
    print('Probability of Rolling a Given Unordered Pair')
    print('(Any Double)')
    print('')
    print('')
    print(f'Value{" ":5}Occurences{" ":5}Observed{" ":6}Expected{" ":6}Absolute Difference', end='')
    print('')
    print(dash)
    for k, v in roll_dict.items():
        if k == 21:
            print('')
            print('')
            print('Probability of Rolling a Given Unordered Pair')
            print('(Any Non-Double)')
            print('')
            print('')
            print(f'Value{" ":5}Occurences{" ":5}Observed{" ":6}Expected{" ":6}Absolute Difference', end='')
            print('')
            print(dash)
        if k in {11,22,33,44,55,66}:
            print(f'{k:<10}{v[0]:<15}{round(v[1],7):<14,.07f}{round(1/36,7):<14,.07f}{abs(round(1/36-v[1],7)):>9,.07f}')
        else: 
            print(f'{k:<10}{v[0]:<15}{round(v[1],7):<14,.07f}{round(2/36,7):<14,.07f}{abs(round(2/36-v[1],7)):>9,.07f}')
    print('')
    print('')
    print('Total Paired Rolls: {:,}'.format(simulation_number))
    print('Nonoccurence Unordered Pairs: {}'.format(nonoccurence_vals))
    print('Total Time: {0:.2f} seconds'.format(time.perf_counter()-tic)) 
    print('')
    print('')
    
    return None
    
probability_of_off_diag_vs_diag_numbers(number_of_simulations)



Probability of Rolling a Given Unordered Pair
(Any Double)


Value     Occurences     Observed      Expected      Absolute Difference
-------------------------------------------------------------------------------
11        69382          0.0277528     0.0277778     0.0000250
22        68924          0.0275696     0.0277778     0.0002082
33        69612          0.0278448     0.0277778     0.0000670
44        69279          0.0277116     0.0277778     0.0000662
55        68979          0.0275916     0.0277778     0.0001862
66        69715          0.0278860     0.0277778     0.0001082


Probability of Rolling a Given Unordered Pair
(Any Non-Double)


Value     Occurences     Observed      Expected      Absolute Difference
-------------------------------------------------------------------------------
21        138226         0.0552904     0.0555556     0.0002652
31        139331         0.0557324     0.0555556     0.0001768
32        138937         0.0555748     0.0555556     0.00001

### Probabilities of Special Features

Now we explore the properties of special combinations categories, including:

   1. Any double
   2. Any non-double
   3. Moving exactly N pips

### Moving Exactly N Pips

To calculate the probability of moving exactly N pips, we need to first make two observations:

1. A pip count is created by summing the die values.
2. When a double occurs, you double the dice, so you double to pip count. For instance, (1,1) allows you to move 1, 2, 3, or 4 pips, while a (5,5) lets you move 5, 10, 15, or 20 pips.

With these, we can easily generate the sample space of possible pip counts (depending on how an individual checker was used during a double) and the rolls that produce them:

|     | 1   | 2   | 3   | 4   | 5   | 6   |
- --- | --- | --- | --- | --- | --- | --- |
| 1 | 11 = 1,2,3,4 | 12 = 3 | 31 = 4 | 14 = 5 | 15 = 6 | 16 = 7 |
| 2 | 21 = 3 | 22 = 2,4,6,8 | 23 = 5 | 24 = 6 | 25 = 7 | 26 = 8 |
| 3 | 31 = 4 | 32 = 5 | 33 = 3,6,9,12 | 34 = 7 | 35 = 8 | 36 = 9 |
| 4 | 41 = 5 | 42 = 6 | 43 = 7 | 44 = 4,8,12,16 | 45 = 9 | 46 = 10 |
| 5 | 51 = 6 | 52 = 7 | 53 = 8 | 54 = 9 | 55 = 5,10,15,20 | 56 = 11 |
| 6 | 61 = 7 | 62 = 8 | 63 = 9 | 64 = 10 | 65 = 11 | 66 = 6,12,18,24 |

This gives us the set of possible pip count values as {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 18, 20, 24}.

Now we can also calculate the roll combinations that allow you to move exactly N pips:

In [12]:
def probability_moving_n_pips(simulation_number):
    '''
    The Monte Carlo simulation for the probability of moving exactly n pips.
    Prints a table of observed probability vs. theoretical probability and
    the rolls that generate the pip count.
    '''
    
    tic = time.perf_counter()
    roll_dict = {}
    prob_dict = dict(zip(range(1,37,1),[i/36 for i in range(1,37,1)]))
    
    def add_to_roll_dict(rd,fn,sn,pc):
        '''
        Adds a value to roll_dict based on the values of the rolls.
        '''
        
        if fn != sn:
            for obj in [fn,sn,pc]:
                if obj in rd.keys():
                    rd[obj][0] += 1 # increment the roll number
                    temp = rd[obj][3]
                    temp.add(int(str(fn)+str(sn)))
                    rd[obj][3] = temp
                else:
                    roll_dict[obj] = [1, # number of rolls
                                     obj, # pip of count
                                     None, # probability column 
                                     set({int(str(fn)+str(sn))}), # roll that generates the pip count
                                     None # expected probability of pip count
                                    ]
        else:
            if pc in rd.keys():
                rd[pc][0] += 1 # increment the roll number
                temp = rd[pc][3]
                temp.add(int(str(fn)+str(sn)))
                rd[pc][3] = temp
            else:
                roll_dict[pc] = [1, # number of rolls
                                 pc, # pip of count
                                 None, # probability column 
                                 set({int(str(fn)+str(sn))}), # roll that generates the pip count
                                 None # expected value
                                ]
        return rd
    
    pip_vals = pip_counts()
    
    for i in range(0,simulation_number,1):
        first_number = np.random.randint(1,7)
        second_number = np.random.randint(1,7)
        if first_number == second_number:
            roll_dict = add_to_roll_dict(roll_dict, # dictionary to add to
                                         first_number, # first number
                                         second_number, # second number
                                         first_number * 1 # pips
                                        ) 
            roll_dict = add_to_roll_dict(roll_dict, # dictionary to add to
                                         first_number, # first number
                                         second_number, # second number
                                         first_number * 2 # pips
                                        ) 
            roll_dict = add_to_roll_dict(roll_dict, # dictionary to add to
                                         first_number, # first number
                                         second_number, # second number
                                         first_number * 3 # pips
                                        ) 
            roll_dict = add_to_roll_dict(roll_dict, # dictionary to add to
                                         first_number, # first number
                                         second_number, # second number
                                         first_number * 4 # pips
                                        ) 
            try:
                pip_vals.remove(first_number)
            except:
                pass
            try:
                pip_vals.remove(2*first_number)
            except:
                pass
            try:
                pip_vals.remove(3*first_number)
            except:
                pass
            try:
                pip_vals.remove(4*first_number)
            except:
                pass
        else:
            roll_dict = add_to_roll_dict(roll_dict, # dictionary to add to
                                         first_number, # first number
                                         second_number, # second number
                                         first_number+second_number # pips
                                        ) 
            try:
                pip_vals.remove(first_number+second_number)
            except:
                pass
            
    for roll in roll_dict.keys():
        roll_dict[roll][2] = roll_dict[roll][0] / simulation_number
        roll_dict[roll][4] = prob_dict[len(roll_dict[roll][3])]
        
    dash = '-' * 135
    
    print('')
    print('')
    print('Moving Exactly N Pips')
    print('')
    print('')
    print(f'Pips {" ":4}Occurences{" ":5}Observed{" ":6}Expected{" ":6}Abs. Diff.{" ":5}Possible Rolls', end='')
    print('')
    print(dash)
    for k, v in sorted(roll_dict.items()):
        print(f'{k:<9}{v[0]:<15}{round(v[2],7):<14,.07f}{round(v[4],7):<14,.07f}{abs(round(v[4]-v[2],7)):<15,.07f}{sorted(v[3])}')
    print('')
    print('')
    print('Total Paired Rolls: {:,}'.format(simulation_number))
    print('Nonoccurence Pip Counts: {}'.format(pip_vals))
    print('Total Time: {0:.2f} seconds'.format(time.perf_counter()-tic)) 
    print('')
    print('')
    
# number of rolls
# pip of count
# probability column 
# roll that generates the pip count
# expected probability of pip count
    
    return None

probability_moving_n_pips(number_of_simulations)



Moving Exactly N Pips


Pips     Occurences     Observed      Expected      Abs. Diff.     Possible Rolls
---------------------------------------------------------------------------------------------------------------------------------------
1        764463         0.3057852     0.3055556     0.0002296      [11, 12, 13, 14, 15, 16, 21, 31, 41, 51, 61]
2        833015         0.3332060     0.3333333     0.0001273      [11, 12, 21, 22, 23, 24, 25, 26, 32, 42, 52, 62]
3        972863         0.3891452     0.3888889     0.0002563      [11, 12, 13, 21, 23, 31, 32, 33, 34, 35, 36, 43, 53, 63]
4        1041896        0.4167584     0.4166667     0.0000917      [11, 13, 14, 22, 24, 31, 34, 41, 42, 43, 44, 45, 46, 54, 64]
5        1042798        0.4171192     0.4166667     0.0004525      [14, 15, 23, 25, 32, 35, 41, 45, 51, 52, 53, 54, 55, 56, 65]
6        1179893        0.4719572     0.4722222     0.0002650      [15, 16, 22, 24, 26, 33, 36, 42, 46, 51, 56, 61, 62, 63, 64, 65, 66]
7        415